<a href="https://colab.research.google.com/github/btoneil2021/Search-Minecraft-Wiki/blob/main/MinecraftWikiScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
HOW TO USE:
1. Type API key below
2. Run
'''
api_key = "[INSERT YOUR API KEY HERE]"

# 1 (and 2 I guess). Scrape Data off Minecraft Wiki

In [ ]:
%%shell
pip install chromedriver_autoinstaller
pip install selenium
pip install beautifulsoup4
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import sys
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import threading
import asyncio

In [ ]:
def get_soup(driver, url):
  driver.get(url)

  driver.implicitly_wait(1)

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  return soup

In [ ]:
# 2 Options
# 1. The page is found and is automatically opened
# 2. The page is not found due to some kind of error and the first link in the search is taken
def search_for_page(driver, search_term):
  '''
  input: search_term - (string) what you're looking for
         driver - (webdriver) the Selenium webdriver instance
  output: url - (string) the url of the page of results you're looking for
  '''
  # I'm just going to be lazy and add an "i" at the end of everything so everything gets kicked to the search page by default
  url = f'https://minecraft.wiki/w/Special:Search?search={search_term.replace(" ", "+")}i'

  soup = get_soup(driver, url)
  url_addition_tag = soup.find("a", {"data-serp-pos": "0"})
  if url_addition_tag and url_addition_tag.has_attr("href"):
      url_addition = url_addition_tag["href"]
      url = f'https://minecraft.wiki{url_addition}'
  else:
      url = driver.current_url 

  return url

In [ ]:
def extract_info_box(soup):
  info_box = soup.find("div", {"class": "notaninfobox"})
  info = {}
  if not info_box:
    return info # Return empty if no info_box

  # Start by Grabbing all the top-of-table information
  name_tag = info_box.find("div", {"class": "mcwiki-header infobox-title"})
  if name_tag:
    info["name"] = name_tag.text.strip()
  
  image_tag_container = info_box.find("div", {"class": "infobox-imagearea animated-container"})
  if image_tag_container:
    image_tag = image_tag_container.find("img")
    if image_tag and image_tag.has_attr('src'):
      info["image_url"] = "https://minecraft.wiki" + image_tag["src"]

  # Ok now we can grab the info from each table section
  info["details"] = {}
  table = info_box.find("table", {"class": "infobox-rows"})
  if table:
    table_rows = table.find_all("tr")
    for row in table_rows:
      key = row.find("th")
      if key is None:
        continue
      value = row.find("td")
      if value is None:
        continue
      info["details"][key.text.strip()] = value.text.strip()

  return info

In [ ]:
def extract_main_paragraph(soup):
  main_paragraph_container = soup.find("div", {"class": "mw-body-content mw-content-ltr"})
  if not main_paragraph_container:
    return [] 
  
  main_paragraph_elements = main_paragraph_container.find_all(["p", "h2", "h3"])

  tables = soup.find_all("table")
  paragraphs_in_tables = []
  for table in tables:
    paragraphs_in_tables.extend(table.find_all("p"))
  
  paragraphs_in_tables_set = set(paragraphs_in_tables)

  # Clearer logic for filtering, ensuring h2/h3 are always included
  filtered = [el for el in main_paragraph_elements if el.name in ['h2', 'h3'] or (el.name == 'p' and el not in paragraphs_in_tables_set)]
  main_paragraph = [el.text.strip() for el in filtered]
  return main_paragraph

In [ ]:
def execute_scraping(search_subject, chrome_options):
  driver = webdriver.Chrome(options=chrome_options)
  driver.implicitly_wait(1)
  try:
    content_url = search_for_page(driver, search_subject)
    soup = get_soup(driver, content_url)
    return soup, driver
  except Exception as e:
    print(f"Error during scraping: {e}")
    if driver:
        driver.quit()
    return None, None

# 3. Throw Everything into ChatGPT

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
from openai import AsyncOpenAI
client = AsyncOpenAI(api_key=api_key)

In [ ]:
async def make_chatgpt_request(prompt_text):
  chat_completion = await client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {
              "role": "user",
              "content": prompt_text
          }
      ]
  )
  return chat_completion.choices[0].message.content

In [ ]:
basic_feature_extraction = '''You are an AI designed to assist with extracting specific features from user requests related to the game Minecraft. Your task is to analyze the user query (delimited by triple backticks) and extract two key pieces of information:

Subject: Identify the main subject of the sentence. This is typically a Minecraft item, entity, block, or structure (full names only).
Summary: Provide a brief summary of the specific information or request that the user wants to know about the subject. Include any information that may be important.
Here are some examples to guide you:

Example 1:
User query: "How do I find diamonds in Minecraft?
Subject: "diamonds"
Summary: "How to find"

Example 2:
User query: "What are the best enchantments for a sword?"
Subject: "sword"
Summary: "best enchantments"

When processing each query, please answer with only the subject and the summary, separated by a backslash.

```

'''

generate_output = '''You are an AI designed to extract specific answers from provided text based on a given summary. Your task is to read the given paragraphs and answer the summary as accurately as possible using the information from the text.
Do not change anything in the summary.

Here is the format you should follow:

Summary: This is a brief statement or question that you need to answer.
Paragraphs: These are the detailed texts from which you will extract the necessary information to answer the summary.
Answer: This is the accurate and concise response to the summary based on the provided paragraphs.
Use the following example as a guide:

Example:
Summary: benefits of a beacon
Paragraphs: Beacons are powerful items in Minecraft that provide various status effects to players within a certain radius. When activated, beacons can grant players speed, haste, resistance, jump boost, or strength. Additionally, a fully powered beacon can give regeneration or a secondary status effect. Beacons are often used in bases to enhance player abilities and provide strategic advantages during combat or resource gathering.
Answer: Beacons provide status effects such as speed, haste, resistance, jump boost, and strength. Fully powered beacons can also give regeneration or a secondary effect, enhancing player abilities and providing strategic advantages in combat and resource gathering.

When processing each query, please answer with only the answer to the summary.

The summary and paragraphs (respectively) are defined below, delimited by triple backticks.
```

'''

In [ ]:
async def prompt(user_query):
  # api_key is global, client (AsyncOpenAI) should be initialized globally or passed if not using global
  
  feature_extraction_prompt = basic_feature_extraction + user_query + "\n```"
  summary_response_text = await make_chatgpt_request(feature_extraction_prompt)
  summary_parts = summary_response_text.strip("\'\"").split("\n")
    
  if len(summary_parts) < 2:
      print(f"Unexpected summary format: {summary_parts}")
      subject = user_query 
      summary_query = "General information"
  else:
      subject = summary_parts[0].strip("Subject: ").strip('"')
      summary_query = summary_parts[1].strip("Summary: ").strip('"')

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument('--disable-gpu')

  driver = None
  soup_obj = None
  info_results = {}
  paragraphs_results = []

  try:
    soup_obj, driver = await asyncio.to_thread(execute_scraping, subject, chrome_options)
    
    if soup_obj:
        info_task = asyncio.to_thread(extract_info_box, soup_obj)
        paragraphs_task = asyncio.to_thread(extract_main_paragraph, soup_obj)
        info_results, paragraphs_results = await asyncio.gather(info_task, paragraphs_task)
    else:
        print(f"Warning: Scraping failed for subject '{subject}', no soup object returned.")
        # Defaults are already set for info_results, paragraphs_results

    info_text_parts = []
    if 'name' in info_results:
        info_text_parts.append(f"Name: {info_results['name']}")
    if 'details' in info_results and isinstance(info_results['details'], dict):
        for k, v in info_results['details'].items():
            info_text_parts.append(f"{k}: {v}")
    info_text = "\n".join(info_text_parts)
    
    paragraphs_text = "\n".join(paragraphs_results)
    
    scraped_content = f"{info_text}\n{paragraphs_text}".strip()

    if not scraped_content:
        print(f"Warning: No information extracted from info_box or paragraphs for subject '{subject}'.")
        return f"Could not extract detailed information for '{subject}' regarding '{summary_query}'. The wiki page might not have the expected structure or the content is missing."

    final_gpt_prompt = generate_output + summary_query + "\n```\n" + scraped_content + "\n```"
    final_answer = await make_chatgpt_request(final_gpt_prompt)
    return final_answer
  except Exception as e:
    print(f"An error occurred in prompt: {e}")
    return "Sorry, I encountered an error processing your request."
  finally:
    if driver:
        await asyncio.to_thread(driver.quit)

In [ ]:
async def main():
  response = await prompt(input("What would you like to ask? "))
  print(response)

In [ ]:
await main()

What would you like to ask? What is dirt?
Summary: What is dirt used for in Minecraft?

Answer: Dirt is primarily used for farming in Minecraft, but due to its abundance, it can also be used as a readily available building block.
